# Getting started with PyEpiDoc

## Loading an EpiDoc file and accessing text

In [1]:
# Load the API
from pyepidoc import EpiDoc, EpiDocCorpus
from pyepidoc.shared.display import print_items
from pyepidoc.epidoc.dom import lang, doc_id, owner_doc

In [2]:
# Load an EpiDoc file without validation
doc = EpiDoc('examples/ISic000001_tokenized.xml')

In [3]:
# Load an EpiDoc file and validate
doc2 = EpiDoc('examples/ISic000001_tokenized.xml', validate_on_load=True)

RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile not

examples/ISic000001_tokenized.xml is a valid EpiDoc file


pile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error tryin

In [4]:
# Print validation result
print(doc2.validation_result)

examples/ISic000001_tokenized.xml is valid


In [5]:
# Print normalized tokens as a string
print(doc.text_normalized)

Dis manibus Zethi vixit annis VI


In [6]:
# Print Leiden tokens as a string
print(doc.text_leiden)


Dis · man(ibus)
 · Zethi
vix(it) · a(nnis) · VI


In [7]:
# Print the translation text
print(doc.translation_text)

## See the XML

In [8]:
print(doc.xml_str)

<?xml version="1.0" encoding="UTF-8"?>
<?xml-model href="http://www.stoa.org/epidoc/schema/latest/tei-epidoc.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"?>
<?xml-model href="../schematron/ircyr-checking.sch" schematypens="http://purl.oclc.org/dsdl/schematron"?>
<TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:xi="http://www.w3.org/2001/XInclude" xml:lang="en">
    <teiHeader>
        <fileDesc>
            <titleStmt>
                <title>Funerary inscription of Zethus</title>
                <editor ref="#JP">Jonathan Prag</editor>
                <principal ref="#JP">Jonathan Prag</principal>
                <funder>John Fell OUP Research Fund</funder>
	   <funder><ref target="https://cordis.europa.eu/project/id/885040">ERC Advanced Grant no.885040</ref></funder>
                <respStmt>
                    <name xml:id="JP" ref="http://orcid.org/0000-0003-3819-8537">Jonathan Prag</name>
                    <resp>original data collection and editin

## Load a corpus

In [9]:
# Load the corpus
corpus = EpiDocCorpus('example_corpus')

In [10]:
# Count the documents
print(corpus.doc_count)

70


In [11]:
# Get the range of dates
print(corpus.datemin, corpus.datemax)

-550 401


## Filter the corpus

In [12]:
# Find all the funerary inscriptions

funerary = corpus.filter_by_textclass(['#function.funerary'])
print(funerary.doc_count)

40


In [13]:
# Chaining filters
catina_funerary = corpus.filter_by_textclass(['#function.funerary']).filter_by_orig_place(['Catina'])

In [14]:
print_items(catina_funerary.ids)


- ISic000002
- ISic000003


In [15]:
# The original corpus is unchanged
print(corpus.doc_count)

70


In [16]:
# Filter by date
imperial_funerary = corpus.filter_by_daterange(start=-1, end=200).filter_by_textclass(['#function.funerary'])

In [17]:
print_items(imperial_funerary.ids)


- ISic000006
- ISic000813
- ISic000819
- ISic000853


In [18]:
# Show the text of particular inscription
print(imperial_funerary.get_doc_by_id('ISic000006').text_leiden)


C(aius) · Iulius · Felix
vixit · ann(os)
Appuleia · Rogata
vixit · ann(os)


## Abbreviations

In [19]:
# Get all the abbreviations in the corpus

expans = corpus.expans
print(len(expans))

162


In [20]:
# Get all the suspensions
print_items(filter(lambda expan: expan.is_suspension == True, expans))


- man(ibus)
- vix(it)
- a(nnis)
- D(is)
- M(anibus)
- s(acrum)
- vix(it)
- Man(ibus)
- vix(it)
- an(nis)
- Herm(es)
- fe(cerunt)
- P(ubli)
- f(ilius)
- P(ublius)
- P(ubli)
- f(ilius)
- P(ubli)
- n(epos)
- f(ilius)
- D(is)
- M(anibus)
- S(acrum)
- M(arcus)
- vix(it)
- ann(is)
- mens(is)
- dieb(us)
- C(aius)
- ann(os)
- ann(os)
- Mag(no)
- Mag(ni)
- f(ilio)
- imp(eratore)
- desig(nato)
- L(ucius)
- L(uci)
- f(ilius)
- leg(atus)
- pr(aetore)
- pr(aetor)
- des(ignatus)
- f(aciendum)
- c(uravit)
- D(is)
- M(anibus)
- fil(ia)
- pient(issimae)
- L(ucius)
- q(uaestor)
- pr(o)
- pr(aetore)
- prov(inciae)
- Sicil(iae)
- leg[(atus)]
- pr(o)
- pr(aetore)
- prov(inciae)
- eiusd(em)
- pr(aetor)
- des[(ignatus)]
- M(arcus)
- ἕνεκ(εν)
- Q(uintus)
- Q(uinti)
- l(ibertus)
- {Ↄ}(mulieris)
- l(ibertus)
- ḍ(e)
- s(ua)
- p(ecunia)
- sacr(um)
- honor(e)
- d(eae)
- d(onum)
- d(edit)
- c(larissimo)
- v(iro)
- Fl(avio)
- Valeri(o)
- v(ir)
- c(larissimus)
- corr(ector)
- prov(inciae)
- λαμπρ(οτάτου)
- ὑπατ(ικὸ

In [21]:
# Get all the Greek suspensions
greek_suspensions = list(filter(
    lambda expan: expan.is_suspension == True and lang(expan) == 'grc', 
    expans))
print_items(greek_suspensions)




- ἕνεκ(εν)
- λαμπρ(οτάτου)
- ὑπατ(ικὸν)
- Θ(εοῖς)
- Κ(αταχθονίοις)
- μῆν(ας)
- χλῖ(ρε)
- μ(ῆνας)
- Θ(εοῖς)
- Κ(αταχθονίοις)
- Θ(εοῖς)
- Κ(αταχθονίοις)
- ἔζησ(εν)
- Θ(εοῖς)
- Κ(αταχθονίοις)
- Κ(λαυδία)
- ἔζ(ησεν)
- μῆ(νας)
- ἡμέ(ρας)
- Ν(ουμέριε)
- μῆν(ας)
- ἡμέρ(ας)
- Κ(υίντος)


In [22]:
# Get the document IDs
ids = map(doc_id, greek_suspensions)
print_items(ids)


- ISic000800
- ISic000811
- ISic000812
- ISic000840
- ISic000840
- ISic000842
- ISic000843
- ISic000844
- ISic000845
- ISic000845
- ISic000852
- ISic000852
- ISic000852
- ISic000853
- ISic000853
- ISic000853
- ISic000853
- ISic000853
- ISic000853
- ISic000854
- ISic000857
- ISic000857
- ISic000858


In [23]:
# Get all the Latin contractions with suspension
latin_contractions_with_suspensions = list(filter(
    lambda expan: expan.is_contraction_with_suspension == True and lang(expan) == 'la', 
    expans))
print_items(latin_contractions_with_suspensions)

print_items(map(doc_id, latin_contractions_with_suspensions))


- co(n)s(ule)
- proco(n)s(uli)

- ISic000007
- ISic000805


In [24]:
# Get the documents

docs = list(map(owner_doc, latin_contractions_with_suspensions))
print_items(map(lambda doc: doc.date_mean, docs))


- -37
- 250


## Tokenizing EpiDoc

In [25]:
doc = EpiDoc('examples/ISic000032_untokenized.xml')
print(doc.tokens)

[]


In [26]:
print(doc.text_xml)

 Marcus · Cornelius · Res Sextus · Clodi 


In [27]:
print(doc.text_leiden)

In [28]:
doc.tokenize()
print(doc.tokens)

Tokenizing ISic000032...
Prettifying ISic000032...
[Element('w': 'Marcus'), Element('w': 'Cornelius'), Element('w': 'Res'), Element('w': 'Sextus'), Element('w': 'Clodi')]


In [29]:
print(doc.text_xml)

 Marcus · Cornelius · Res Sextus · Clodi 


In [30]:
print(doc.text_leiden)


M(arcus) · Cornelius · Ṛẹṣ[-?-]
Sex(tus) · Clodị[-?-]


In [31]:
print(doc.text_normalized)

Marcus Cornelius Res Sextus Clodi


## Setting IDs

In [32]:
print(doc.editions()[0].xml_str)

<div xmlns="http://www.tei-c.org/ns/1.0" xmlns:xi="http://www.w3.org/2001/XInclude" type="edition" xml:space="preserve" xml:lang="la">
                <ab>
                    <lb n="1"/><w><expan><abbr>M</abbr><ex>arcus</ex></expan></w> <g ref="#interpunct">·</g> <w>Cornelius</w> <g ref="#interpunct">·</g> <w><unclear>Res</unclear></w> <gap reason="lost" extent="unknown" unit="character"/>
                    <lb n="2"/><w><expan><abbr>Sex</abbr><ex>tus</ex></expan></w> <g ref="#interpunct">·</g> <w>Clod<unclear>i</unclear></w> <gap reason="lost" extent="unknown" unit="character"/>
                </ab>
            </div>
            



In [33]:
print_items(doc.ids)

In [34]:
doc.set_ids()
print_items(doc.ids)


- ADUAK
- ADUAU
- ADUAe
- ADUAo
- ADUAy
- ADUAΙ
- ADUAΤ
- ADUAε
- ADUAο
- ADUBA
- ADUBK
- ADUBU
- ADUBe
- ADUBo
- ADUBy
- ADUBΙ
- ADUBΤ
- ADUBε
- ADUBο
- ADUCA


In [35]:
print(doc.editions()[0].xml_str)
      

<div xmlns="http://www.tei-c.org/ns/1.0" xmlns:xi="http://www.w3.org/2001/XInclude" type="edition" xml:space="preserve" xml:lang="la">
                <ab>
                    <lb n="1" xml:id="ADUAK"/><w xml:id="ADUAU"><expan xml:id="ADUAe"><abbr xml:id="ADUAo">M</abbr><ex xml:id="ADUAy">arcus</ex></expan></w> <g ref="#interpunct" xml:id="ADUAΙ">·</g> <w xml:id="ADUAΤ">Cornelius</w> <g ref="#interpunct" xml:id="ADUAε">·</g> <w xml:id="ADUAο"><unclear xml:id="ADUBA">Res</unclear></w> <gap reason="lost" extent="unknown" unit="character" xml:id="ADUBK"/>
                    <lb n="2" xml:id="ADUBU"/><w xml:id="ADUBe"><expan xml:id="ADUBo"><abbr xml:id="ADUBy">Sex</abbr><ex xml:id="ADUBΙ">tus</ex></expan></w> <g ref="#interpunct" xml:id="ADUBΤ">·</g> <w xml:id="ADUBε">Clod<unclear xml:id="ADUBο">i</unclear></w> <gap reason="lost" extent="unknown" unit="character" xml:id="ADUCA"/>
                </ab>
            </div>
            



In [37]:
print(doc.validate()[1])

/data/programming/python/pyepidoc/PyEpiDoc/src/pyepidoc/tei-epidoc.rng is valid EpiDoc according to the RelaxNG schema


RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile notAllowed
RNG internal error trying to compile not